In [118]:
import pandas as pd
import numpy as np
import matplotlib as plt
import statsmodels.formula.api as sm

## Import Data

In [119]:
df = pd.read_csv('data/sample_df.csv')
df.set_index('Month',inplace=True, drop=True)

In [120]:
# df['TV'].index

In [121]:
df.head()

,Volume,TV,Digital,Other,Coverage,RetailPrice,Temp
Month,,,,,,,
2016/01/01,4.679758e+06,24727,4867,0,0.221294,61.822051,6.222581
2016/02/01,5.612667e+06,102245,19383,11877,0.219405,62.132821,7.047126
2016/03/01,7.081166e+06,111393,16725,11987,0.219454,61.820769,10.459140
2016/04/01,8.366490e+06,64051,18815,0,0.217712,61.524615,15.961111
2016/05/01,1.239568e+07,134791,26876,0,0.213228,61.276410,20.646237


## Descriptive Analysis

## Adstock Transformation

### Apply Adstock

In [122]:
#lag and decay and power
def to_adstock(media,lag,decay,alpha):
    
    n = len(media)
    with_AS = np.zeros(n)
    
    for i in range(int(lag),n):
        with_AS[i] = media[i-lag]**alpha + decay*with_AS[i-1]
        
    return (with_AS)

## Modelling

### Model fitting

#####   Left: name the pd.Series with Column names following the rule
1. TV_1L: TV_xL x as the number of Lags
2. TV_AS60: AS means astock, this means taking decay of 60% from previous one
3. TV_Alpha90: means taking power curve transformation and, x**aplha, alpha = 0.9

In [123]:
def fit_model1(data,a,b,c,d,e,f,g,h,i):
    
    tv_ads=to_adstock(data['TV'],a,b,c)
    digital_ads=to_adstock(data['Digital'],d,e,f)
    other_ads=to_adstock(data['Other'],g,h,i)
    sales=data['Volume']
    temp=data['Temp']
    coverage=data['Coverage']
    retailprice=data['RetailPrice']
    x_ad=pd.concat([data['TV'],pd.Series(tv_ads,index=data['TV'].index),data['Digital'],pd.Series(digital_ads,index=data['TV'].index),data['Other'],pd.Series(other_ads,index=data['TV'].index),temp,coverage,retailprice,pd.Series(sales,index=data['TV'].index)],axis =1, join_axes=[data['TV'].index])

    # modelling
    
    models=sm.ols(formula='sales ~ tv_ads+digital_ads+other_ads+temp+coverage+retailprice',data=x_ad).fit()
    
    return models

In [124]:
x = fit_model1(df,2,0.6,0.9,2,0.6,1,2,0.6,1)

In [125]:
x.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sales   R-squared:                       0.828
Model:                            OLS   Adj. R-squared:                  0.781
Method:                 Least Squares   F-statistic:                     17.64
Date:                Wed, 20 Feb 2019   Prob (F-statistic):           2.17e-07
Time:                        17:00:43   Log-Likelihood:                -464.91
No. Observations:                  29   AIC:                             943.8
Df Residuals:                      22   BIC:                             953.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -1.283e+07    8.7e+06     -1.475      0.154   -3.09e+07    5.21e+06
tv_ads        -29.9279     19.130     -1.564      0.132     -69.601       9.745
digital_ads    10.7572     31.412      0.342      0.735     -54.386      75.901
other_ads      93.6585     66.616      1.406      0.174     -44.495     231.812
temp         5.084e+05   1.11e+05      4.566      0.000    2.77e+05    7.39e+05
coverage     7.207e+07   4.66e+07      1.547      0.136   -2.46e+07    1.69e+08
retailprice -3.494e+04   4.02e+04     -0.869      0.394   -1.18e+05    4.84e+04
==============================================================================
Omnibus:                        0.745   Durbin-Watson:                   1.448
Prob(Omnibus):                  0.689   Jarque-Bera (JB):                0.577
Skew:                           0.328   Prob(JB):                        0.750
Kurtosis:                       2.786   Cond. No.                     1.05e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.05e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [126]:
predicted = x.predict()

In [127]:
true= df['Volume'].values

##### Check MAPE

In [128]:
from sklearn.utils.validation import check_array

In [130]:
def mean_absolute_percentage_error(y_true, y_pred): 
    MAPE = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return MAPE

In [131]:
MAPE = mean_absolute_percentage_error(true,predicted)

In [132]:
MAPE

22.334310946533947

1. Too Large, Need to check for seasonality and multicollearity
2. VIF ?

In [ ]:
def model(data):
    # Run OLS regression, print summary and return results
    
    lag_in =1
    decay_in= -0.1
    power_in = - 0.1
    
    tv_lag = list(np.arange(1,4, lag_in))
    tv_decay = list(np.arange(0.9, 0.8, decay_in))
    tv_power = list (np.arange(0.9, 0.8, power_in))
    digital_lag = list(np.arange(1,4, lag_in))
    digital_decay = list(np.arange(0.9, 0.8, decay_in))
    digital_power = list (np.arange(0.9, 0.8,power_in))
    other_lag = list(np.arange(1,4, lag_in))
    other_decay = list(np.arange(0.9, 0.8, decay_in))
    other_power = list (np.arange(0.9, 0.8, power_in))


    # best=[]
    result=[]

    for a in tv_lag:
        for b in tv_decay:
            for c in tv_power:
                for d in digital_lag:
                    for e in digital_decay:
                        for f in digital_power:
                            for g in tv_lag:
                                for h in tv_decay:
                                    for i in tv_power:
                                        currentbest=[]
                                        final=[]
                                        final1=[]
  
                                        train_model=fit_model1(data,a,b,c,d,e,f,g,h,i)
                                        final.append(train_model)
        
                                        # 想在这个地方加入下面那个forward selection 达到的目的,在下一个cell里面说明
                                        for i in final:
                                            
                            
        
    return result

1. 接上面 要构筑一个线性模型 'sales ~ tv_ads+digital_ads+other_ads+(temp+coverage+retailprice)'
2. tv,digital,other 必须加入，并且通过调用上面的循环，给他们赋予不同的参数(lag,decay,power).
3. 剩下的三个变量 temp retailprice coverage， 想要选择最优的加入。下面有网上forward selection的method
4. 比如果三个必须加入的变量每个lag数3 decay 2 power 2，再加上剩下三个变量的各种组合 应该是一共构建 (3*2*2)^3*6 = 34992个模型选最优？

### Ref.

#### 1. Forward selection (for vars)

参考 https://planspace.org/20150423-forward_selection_with_statsmodels/

In [ ]:
import statsmodels.formula.api as smf

def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model